In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/csq-pred/1-11.xlsx
/kaggle/input/csq-validate/12.xlsx


In [2]:
import warnings
warnings.filterwarnings('ignore')

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn import preprocessing
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split

from sklearn import svm 
from keras.layers import Dense, BatchNormalization, Dropout, LSTM
from keras.models import Sequential
from keras import callbacks
from sklearn.metrics import precision_score, recall_score, confusion_matrix, classification_report, accuracy_score, f1_score

2024-04-02 15:14:54.026582: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-04-02 15:14:54.026772: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-04-02 15:14:54.191501: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


#  锚泊状态

In [3]:
data_df = pd.read_excel("../input/csq-pred/1-11.xlsx")
validate_df = pd.read_excel("../input/csq-validate/12.xlsx")

all_data = pd.concat([data_df, validate_df], ignore_index=True)
anchor_data_df = data_df[data_df['状态'] == '锚泊']
anchor_all_data = all_data[all_data['状态'] == '锚泊']


X = anchor_all_data[['风向', '浪向', '风级', '浪级']]
y = anchor_all_data[['重油消耗量', '轻油消耗量']]

In [4]:
# 对风向和浪向进行独热编码
X = pd.get_dummies(X, columns=['风向', '浪向'])

In [5]:
X_train = X[0:len(anchor_data_df)]
X_train = np.asarray(X_train).astype('float32')

y_train = y[0:len(anchor_data_df)]
y_train = np.asarray(y_train).astype('float32')

In [6]:
X_test = X[len(anchor_data_df)::]
X_test = np.asarray(X_test).astype('float32')

y_test = y[len(anchor_data_df)::]
y_test = np.asarray(y_test).astype('float32')

In [7]:
X_train.shape, X_test.shape, y_train.shape, y_test.shape

((207, 16), (17, 16), (207, 2), (17, 2))

In [8]:
# 创建 EarlyStopping 回调对象
early_stopping = callbacks.EarlyStopping(
    min_delta=0.001,  # 最小改善量
    patience=20,  # 在停止之前等待的轮数
    restore_best_weights=True  # 恢复在验证集上表现最好的权重
)

# 创建神经网络模型
model = Sequential()

# 添加层
model.add(Dense(units=16, kernel_initializer='uniform', activation='relu', input_dim=X_train.shape[1]))
model.add(Dense(units=8, kernel_initializer='uniform', activation='relu'))
model.add(Dropout(0.25))
model.add(Dense(units=8, kernel_initializer='uniform', activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(units=2, kernel_initializer='uniform', activation = 'relu'))  # 输出层改为2个单元

# 编译模型
model.compile(optimizer='adam', loss='mean_squared_error', metrics = ['accuracy'])  # 使用均方误差作为损失函数

# 训练模型
history = model.fit(X_train, y_train, batch_size=25, epochs=800, callbacks=[early_stopping], validation_split=0.25)

Epoch 1/800
7/7 ━━━━━━━━━━━━━━━━━━━━ 2s 39ms/step - accuracy: 0.7412 - loss: 8.9682 - val_accuracy: 1.0000 - val_loss: 8.5597
Epoch 2/800
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - accuracy: 0.8248 - loss: 9.4884 - val_accuracy: 1.0000 - val_loss: 8.5223
Epoch 3/800
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - accuracy: 0.8315 - loss: 9.6114 - val_accuracy: 1.0000 - val_loss: 8.4792
Epoch 4/800
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - accuracy: 0.8867 - loss: 8.7883 - val_accuracy: 1.0000 - val_loss: 8.4269
Epoch 5/800
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - accuracy: 0.9165 - loss: 8.4184 - val_accuracy: 1.0000 - val_loss: 8.3588
Epoch 6/800
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - accuracy: 0.9275 - loss: 8.8867 - val_accuracy: 1.0000 - val_loss: 8.2673
Epoch 7/800
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - accuracy: 0.9588 - loss: 9.3360 - val_accuracy: 1.0000 - val_loss: 8.1390
Epoch 8/800
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step - accuracy: 0.9976 - loss: 8.2037 - val_accuracy: 1.0000 - val_loss: 7.958

In [9]:
loss, accuracy = model.evaluate(X_test, y_test)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 25ms/step - accuracy: 1.0000 - loss: 0.2937


In [10]:
predictions = model.predict(X_test)
predictions,y_test

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 68ms/step


(array([[3.514273 , 0.       ],
        [3.514273 , 0.       ],
        [3.514273 , 0.       ],
        [3.514273 , 0.       ],
        [3.514273 , 0.       ],
        [3.5017006, 0.       ],
        [3.5017006, 0.       ],
        [3.5017006, 0.       ],
        [3.5017006, 0.       ],
        [3.8420563, 0.       ],
        [3.8420563, 0.       ],
        [4.4841003, 0.       ],
        [3.5017006, 0.       ],
        [3.5017006, 0.       ],
        [3.5017006, 0.       ],
        [3.5017006, 0.       ],
        [3.5017006, 0.       ]], dtype=float32),
 array([[3.48 , 0.1  ],
        [3.8  , 0.1  ],
        [3.68 , 0.1  ],
        [3.51 , 0.1  ],
        [3.1  , 0.1  ],
        [3.1  , 0.1  ],
        [5.   , 0.1  ],
        [4.34 , 0.1  ],
        [3.1  , 0.1  ],
        [3.1  , 0.1  ],
        [3.1  , 0.1  ],
        [3.418, 0.1  ],
        [3.1  , 0.1  ],
        [5.34 , 0.1  ],
        [3.1  , 0.1  ],
        [3.1  , 0.1  ],
        [3.1  , 0.1  ]], dtype=float32))

# 航行状态

In [11]:
anchor_data_df = data_df[data_df['状态'] == '航行24h']
anchor_all_data = all_data[all_data['状态'] == '航行24h']


X = anchor_all_data[['实测时速', '推进器转速 r/min', '风向', '浪向', '风级', '浪级', '航向', ]]
y = anchor_all_data[['重油消耗量', '轻油消耗量']]

In [12]:
# 对风向和浪向进行独热编码
X = pd.get_dummies(X, columns=['风向', '浪向'])

In [13]:
# 将航向的度数改为数值

headings = X['航向']

# 去除度数符号并转换为整数
headings_numeric = headings.str.replace('°', '').astype(int)


# 替换原始数据中的航向列
X['航向'] = headings_numeric
X

,实测时速,推进器转速 r/min,风级,浪级,航向,风向_NE,风向_NW,风向_SE,风向_SW,浪向_NE,浪向_NW,浪向_SE,浪向_SW
8,12.0,100.0,5,4,62,False,True,False,False,False,True,False,False
17,11.6,99.0,6,5,285,False,True,False,False,False,True,False,False
24,13.0,102.0,4,3,107,False,True,False,False,False,True,False,False
25,12.7,102.0,5,3,51,False,True,False,False,False,True,False,False
26,12.7,102.0,5,3,13,False,True,False,False,False,True,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...
345,9.4,99.0,7,5,12,True,False,False,False,True,False,False,False
346,10.0,99.0,6,4,335,True,False,False,False,True,False,False,False
356,5.0,99.0,8,5,352,False,True,False,False,False,True,False,False
357,7.5,99.0,8,5,352,False,True,False,False,False,True,False,False


In [14]:
X_train = X[0:len(anchor_data_df)]
X_train = np.asarray(X_train).astype('float32')

y_train = y[0:len(anchor_data_df)]
y_train = np.asarray(y_train).astype('float32')

In [15]:
X_train, y_train

(array([[ 12. , 100. ,   5. , ...,   1. ,   0. ,   0. ],
        [ 11.6,  99. ,   6. , ...,   1. ,   0. ,   0. ],
        [ 13. , 102. ,   4. , ...,   1. ,   0. ,   0. ],
        ...,
        [ 12. ,  99. ,   4. , ...,   0. ,   0. ,   0. ],
        [ 13.7, 101. ,   4. , ...,   0. ,   1. ,   0. ],
        [ 13. , 101. ,   4. , ...,   0. ,   1. ,   0. ]], dtype=float32),
 array([[22.54,  0.1 ],
        [22.91,  0.1 ],
        [20.99,  0.1 ],
        [20.6 ,  0.1 ],
        [20.6 ,  0.1 ],
        [22.92,  0.1 ],
        [22.92,  0.1 ],
        [22.  ,  0.1 ],
        [22.  ,  0.1 ],
        [22.  ,  0.1 ],
        [22.  ,  0.1 ],
        [22.92,  0.1 ],
        [22.  ,  0.1 ],
        [16.57,  0.08],
        [20.6 ,  0.1 ],
        [21.5 ,  0.1 ],
        [21.5 ,  0.1 ],
        [22.  ,  0.1 ],
        [23.1 ,  0.1 ],
        [23.1 ,  0.1 ],
        [23.1 ,  0.1 ],
        [23.1 ,  0.1 ],
        [23.1 ,  0.1 ],
        [23.1 ,  0.1 ],
        [21.77,  0.  ],
        [16.46,  0.1 ],
    

In [16]:
X_test = X[len(anchor_data_df)::]
X_test = np.asarray(X_test).astype('float32')

y_test = y[len(anchor_data_df)::]
y_test = np.asarray(y_test).astype('float32')

In [17]:
X_test, y_test

(array([[ 12. , 101. ,   4. ,   3. ,  38. ,   1. ,   0. ,   0. ,   0. ,
           1. ,   0. ,   0. ,   0. ],
        [ 12.4, 101. ,   4. ,   3. ,   2. ,   1. ,   0. ,   0. ,   0. ,
           1. ,   0. ,   0. ,   0. ],
        [ 11.9, 101. ,   5. ,   4. , 355. ,   1. ,   0. ,   0. ,   0. ,
           1. ,   0. ,   0. ,   0. ],
        [ 11.9, 101. ,   5. ,   4. , 355. ,   1. ,   0. ,   0. ,   0. ,
           1. ,   0. ,   0. ,   0. ],
        [  9.3,  99. ,   7. ,   5. ,  16. ,   1. ,   0. ,   0. ,   0. ,
           1. ,   0. ,   0. ,   0. ],
        [  8.9,  99. ,   7. ,   5. ,  17. ,   1. ,   0. ,   0. ,   0. ,
           1. ,   0. ,   0. ,   0. ],
        [  8.3,  99. ,   8. ,   6. ,  18. ,   1. ,   0. ,   0. ,   0. ,
           1. ,   0. ,   0. ,   0. ],
        [  9.4,  99. ,   7. ,   5. ,  12. ,   1. ,   0. ,   0. ,   0. ,
           1. ,   0. ,   0. ,   0. ],
        [ 10. ,  99. ,   6. ,   4. , 335. ,   1. ,   0. ,   0. ,   0. ,
           1. ,   0. ,   0. ,   0. ],
        [ 

In [18]:
# 创建 EarlyStopping 回调对象
early_stopping = callbacks.EarlyStopping(
    min_delta=0.001,  # 最小改善量
    patience=20,  # 在停止之前等待的轮数
    restore_best_weights=True  # 恢复在验证集上表现最好的权重
)

# 创建神经网络模型
model = Sequential()

# 添加层
model.add(Dense(units=16, kernel_initializer='uniform', activation='relu', input_dim=X_train.shape[1]))
model.add(Dense(units=8, kernel_initializer='uniform', activation='relu'))
model.add(Dropout(0.25))
model.add(Dense(units=8, kernel_initializer='uniform', activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(units=2, kernel_initializer='uniform', activation = 'relu'))  # 输出层改为2个单元

# 编译模型
model.compile(optimizer='adam', loss='mean_squared_error', metrics = ['accuracy'])  # 使用均方误差作为损失函数

# 训练模型
history = model.fit(X_train, y_train, batch_size=25, epochs=800, callbacks=[early_stopping], validation_split=0.25)

Epoch 1/800
3/3 ━━━━━━━━━━━━━━━━━━━━ 2s 103ms/step - accuracy: 0.9879 - loss: 248.1402 - val_accuracy: 1.0000 - val_loss: 238.3988
Epoch 2/800
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step - accuracy: 1.0000 - loss: 246.1524 - val_accuracy: 1.0000 - val_loss: 238.3097
Epoch 3/800
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step - accuracy: 1.0000 - loss: 246.4231 - val_accuracy: 1.0000 - val_loss: 238.2041
Epoch 4/800
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step - accuracy: 0.9779 - loss: 246.3900 - val_accuracy: 1.0000 - val_loss: 238.0750
Epoch 5/800
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step - accuracy: 1.0000 - loss: 244.2689 - val_accuracy: 1.0000 - val_loss: 237.9125
Epoch 6/800
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step - accuracy: 1.0000 - loss: 248.9892 - val_accuracy: 1.0000 - val_loss: 237.6998
Epoch 7/800
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step - accuracy: 1.0000 - loss: 250.1446 - val_accuracy: 1.0000 - val_loss: 237.4149
Epoch 8/800
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step - accuracy: 1.0000 - loss: 246.7829 - v

In [19]:
predictions = model.predict(X_test)
predictions,y_test

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 64ms/step


(array([[13.211333 ,  0.       ],
        [12.0246725,  0.       ],
        [24.175718 ,  0.       ],
        [24.175718 ,  0.       ],
        [12.465879 ,  0.       ],
        [12.461126 ,  0.       ],
        [12.620269 ,  0.       ],
        [12.339415 ,  0.       ],
        [23.209352 ,  0.       ],
        [23.475409 ,  0.       ],
        [23.717861 ,  0.       ],
        [24.073725 ,  0.       ]], dtype=float32),
 array([[22.  ,  0.1 ],
        [21.99,  0.1 ],
        [22.  ,  0.1 ],
        [22.  ,  0.1 ],
        [21.99,  0.1 ],
        [21.99,  0.1 ],
        [21.99,  0.1 ],
        [22.  ,  0.1 ],
        [21.84,  0.1 ],
        [14.46,  0.1 ],
        [21.49,  0.1 ],
        [21.49,  0.1 ]], dtype=float32))

In [20]:
X_train, y_train

(array([[ 12. , 100. ,   5. , ...,   1. ,   0. ,   0. ],
        [ 11.6,  99. ,   6. , ...,   1. ,   0. ,   0. ],
        [ 13. , 102. ,   4. , ...,   1. ,   0. ,   0. ],
        ...,
        [ 12. ,  99. ,   4. , ...,   0. ,   0. ,   0. ],
        [ 13.7, 101. ,   4. , ...,   0. ,   1. ,   0. ],
        [ 13. , 101. ,   4. , ...,   0. ,   1. ,   0. ]], dtype=float32),
 array([[22.54,  0.1 ],
        [22.91,  0.1 ],
        [20.99,  0.1 ],
        [20.6 ,  0.1 ],
        [20.6 ,  0.1 ],
        [22.92,  0.1 ],
        [22.92,  0.1 ],
        [22.  ,  0.1 ],
        [22.  ,  0.1 ],
        [22.  ,  0.1 ],
        [22.  ,  0.1 ],
        [22.92,  0.1 ],
        [22.  ,  0.1 ],
        [16.57,  0.08],
        [20.6 ,  0.1 ],
        [21.5 ,  0.1 ],
        [21.5 ,  0.1 ],
        [22.  ,  0.1 ],
        [23.1 ,  0.1 ],
        [23.1 ,  0.1 ],
        [23.1 ,  0.1 ],
        [23.1 ,  0.1 ],
        [23.1 ,  0.1 ],
        [23.1 ,  0.1 ],
        [21.77,  0.  ],
        [16.46,  0.1 ],
    

In [21]:
X_test, y_test

(array([[ 12. , 101. ,   4. ,   3. ,  38. ,   1. ,   0. ,   0. ,   0. ,
           1. ,   0. ,   0. ,   0. ],
        [ 12.4, 101. ,   4. ,   3. ,   2. ,   1. ,   0. ,   0. ,   0. ,
           1. ,   0. ,   0. ,   0. ],
        [ 11.9, 101. ,   5. ,   4. , 355. ,   1. ,   0. ,   0. ,   0. ,
           1. ,   0. ,   0. ,   0. ],
        [ 11.9, 101. ,   5. ,   4. , 355. ,   1. ,   0. ,   0. ,   0. ,
           1. ,   0. ,   0. ,   0. ],
        [  9.3,  99. ,   7. ,   5. ,  16. ,   1. ,   0. ,   0. ,   0. ,
           1. ,   0. ,   0. ,   0. ],
        [  8.9,  99. ,   7. ,   5. ,  17. ,   1. ,   0. ,   0. ,   0. ,
           1. ,   0. ,   0. ,   0. ],
        [  8.3,  99. ,   8. ,   6. ,  18. ,   1. ,   0. ,   0. ,   0. ,
           1. ,   0. ,   0. ,   0. ],
        [  9.4,  99. ,   7. ,   5. ,  12. ,   1. ,   0. ,   0. ,   0. ,
           1. ,   0. ,   0. ,   0. ],
        [ 10. ,  99. ,   6. ,   4. , 335. ,   1. ,   0. ,   0. ,   0. ,
           1. ,   0. ,   0. ,   0. ],
        [ 

In [22]:
anchor_data_df

,日期,实测时速,推进器转速 r/min,风向,浪向,风级,浪级,航向,重油消耗量,轻油消耗量,状态
8,2022.1.9,12.0,100.0,NW,NW,5,4,062°,22.54,0.10,航行24h
17,2022.1.18,11.6,99.0,NW,NW,6,5,285°,22.91,0.10,航行24h
24,2022.1.25,13.0,102.0,NW,NW,4,3,107°,20.99,0.10,航行24h
25,2022.1.26,12.7,102.0,NW,NW,5,3,051°,20.60,0.10,航行24h
26,2022.1.27,12.7,102.0,NW,NW,5,3,013°,20.60,0.10,航行24h
...,...,...,...,...,...,...,...,...,...,...,...
308,2022.11.5,12.0,99.0,NE,NE,4,3,216°,22.88,0.11,航行24h
309,2022.11.6,13.0,101.0,NE,NE,5,4,234°,21.98,0.10,航行24h
313,2022.11.10,12.0,99.0,NE,NE,4,3,320°,21.92,0.10,航行24h
329,2022.11.26,13.7,101.0,SE,SE,4,3,146°,20.59,0.10,航行24h
